# Universal Adversarial Examples on SMAL with PointNet classifier

First, we need to import the necessary modules.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

     |████████████████████████████████| 2.6MB 5.9MB/s 
     |████████████████████████████████| 1.5MB 7.3MB/s 
     |████████████████████████████████| 215kB 9.9MB/s 
     |████████████████████████████████| 235kB 16.1MB/s 
     |████████████████████████████████| 2.2MB 18.7MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 


In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
## Built-in libraries.
import sys
import os 

## Third party libraries.
import matplotlib.pyplot as plt 
import numpy as np
import torch 

REPO_ROOT  = '/content/drive/My Drive/SpectralAdversarialAttacks'  ####
DEVICE     = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
SRC_DIR    = os.path.join(REPO_ROOT, "src")

## Repository modules.
sys.path.insert(0, SRC_DIR)
import models
import dataset
from utils import visualize, compare, visualize_and_compare

Load pre-trained PointNet classifier:

In [6]:
from models.pointnet import *

## SMAL.
SMAL            = os.path.join(REPO_ROOT, "datasets/smal")
SMAL_PARAMETERS = os.path.join(REPO_ROOT, "model_data/smal-pointnet.pt")
PARAMS_FILE     = SMAL_PARAMETERS

# Encoder.
LATENT_SPACE = 128
ENC = SimplePointNet(latent_dimensionality=LATENT_SPACE,
                     convolutional_output_dim=512,
                     conv_layer_sizes=[32, 128, 256],
                     fc_layer_sizes=[512, 256, 128],
                     transformer_positions=[0]).to(DEVICE)

# Classifier.
CLA = nn.Sequential(nn.Linear(LATENT_SPACE, 64), nn.ReLU(), nn.Linear(64,5)).to(DEVICE)

## Model.
model = SMALClassifier(ENC, CLA, LATENT_SPACE).to(DEVICE)
model.load_state_dict(torch.load(PARAMS_FILE, map_location=torch.device("cpu")))

<All keys matched successfully>

Load a set of shapes from SMAL dataset:

In [7]:
label_dic = {0: 'lions',
             1: 'cows',
             2: 'dogs',
             3: 'hippos',
             4: 'horses'
}

custom_list = [1,2,3,4,5,6,7,8]
customdata  = dataset.SmalDataset(SMAL, device=DEVICE, train=False, test=False, custom=True, custom_list=custom_list, transform_data=False)
n_shapes    = len(customdata)

## Configure adversarial attack

In [8]:
from universal.builder import Builder

## Parameters used to search the adversarial sample.
params = {
    Builder.USETQDM    : 'notebook',
    Builder.MIN_IT     : 1000,   #<- Number of iterations for the adversarial example computation (common value: 1000).
    Builder.LEARN_RATE : 1e-4,   #<- Learning rate used during optimization.
    # Coefficients.
    Builder.ADV_COEFF  : 0.07,   #<- Starting coefficient applied to the adversarial loss (updated via exponential search).
    Builder.REG_COEFF  : 0,      #<- Coefficient applied to the regularization term.
    Builder.SIM_COEFF  : 0,      #<- Coefficient applied to the cross similarity term (gamma).
    Builder.ISO_COEFF  : 10000,  #<- Coefficient applied to the isospectralization term.
    Builder.ADV_LOSS_K : 0.5,
    # Eigenvalues number.
    Builder.EIGS_ISOSP_NUM     : 61, #<- Number of eigenvalues for isospectralization.
    Builder.EIGS_BANDWIDTH_NUM : 20  #<- Number of eigenvalues for low bandwith distorsion.
}


## Configure adversarial example components using builder.
#--------------------------------------------------------
builder = Builder(search_iterations=1)
builder.set_classifier(model)
builder.set_dataset(customdata)
builder.set_target(None)
builder.set_template_index(0)

## Set perturbation.
from universal.perturbation import UniversalPerturbation
builder.set_perturbation(UniversalPerturbation)

## Set losses.
from universal.loss import *
# Similarity Losses.
builder.set_similarity_loss(ZeroLoss)
builder.set_cross_similarity_loss(ZeroLoss)
# Adversarial Loss.
builder.set_adversarial_loss(UniversalAdversarialLoss)
# Isospectralization loss.
builder.set_isospectralization_loss(UniversalIsospec)
# Regularization Loss
builder.set_regularization_loss(ZeroLoss)

## Perform adversarial attack

In [9]:
adv_ex = builder.build(**params)

[0,0.07] ; c=0.07


## Visualize results
Check misclassification:

In [10]:
## Print original and final logits and compute success list.

cmodel = lambda x : model(x.float())
success_list = []

for shape_index in range(len(customdata)):
    
    print('\nShape ', custom_list[shape_index])    

    perturbed_pos_i = adv_ex.perturbed_positions_i(shape_index)
    out: torch.Tensor = cmodel(perturbed_pos_i)
    _, lbl_prediction = out.max(dim=-1)
    print('Perturbed logits : ', out.detach().cpu().numpy())

    pos_i = customdata[shape_index].pos
    out: torch.Tensor = cmodel(pos_i)
    _, lbl_original = out.max(dim=-1)
    print('Original logits  : ', out.detach().cpu().numpy())

    success = (lbl_prediction != lbl_original).item()
    success_list.append(success)

    print("Successful" if not int(lbl_prediction) == int(lbl_original) else "Unsuccessful",
          "[prediction: ", int(lbl_prediction),
          ", original: ", int(lbl_original), "]")


Shape  1
Perturbed logits :  [-1.6415565  -0.7147267  -2.8378856   0.66688406  0.03608184]
Original logits  :  [-2.4567692  -0.55528164 -3.6982324  -0.32199374  0.926794  ]
Successful [prediction:  3 , original:  4 ]

Shape  2
Perturbed logits :  [-2.5797193  -0.98412335 -3.9156697   0.99172974  0.33212772]
Original logits  :  [-5.910388    0.46549127 -7.3217893  -5.818109    4.6407475 ]
Successful [prediction:  3 , original:  4 ]

Shape  3
Perturbed logits :  [-2.8767593   1.3779333  -3.9150536  -1.3701383   0.87792987]
Original logits  :  [-3.1061332   0.01541001 -4.03901    -1.1888628   1.5800855 ]
Successful [prediction:  1 , original:  4 ]

Shape  4
Perturbed logits :  [ 0.10892587 -0.2316095  -2.087329   -0.648939   -0.41554493]
Original logits  :  [-4.6695056  0.5625308 -6.034156  -5.160281   3.748162 ]
Successful [prediction:  0 , original:  4 ]

Shape  5
Perturbed logits :  [-1.3164202 -0.8820927 -2.7519603  0.7515089 -0.2537228]
Original logits  :  [-2.677364   -0.28105578 -

[autoreload of utils.eigendecomposition failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
RecursionError: maximum recursion depth exceeded
]


Visualize an adversarial shape:

In [11]:
shape_index = 0 # Shape to visualize.

k_evals = params[Builder.EIGS_ISOSP_NUM]

## Original shape.
pos = customdata[shape_index].pos
faces = adv_ex.faces(shape_index)

## Load perturbed shape.
perturbed_pos = adv_ex.perturbed_positions_i(shape_index)

## Rotate meshes for visualization.
pos_i_rotated           = pos.clone()
pos_i_rotated[:,1]      = -1 * pos_i_rotated[:,1]

perturbed_pos_i_rotated      = perturbed_pos.clone()
perturbed_pos_i_rotated[:,1] = -1 * perturbed_pos_i_rotated[:,1]

print(success_list[shape_index])
visualize_and_compare(perturbed_pos_i_rotated, faces, pos_i_rotated, faces, (pos-perturbed_pos).norm(p=2,dim=-1))

True


# Generalization on a new shape
Choose a new shape:

In [12]:
def common_member(a, b): 
    a_set = set(a) 
    b_set = set(b) 
    if (a_set & b_set): 
        return True 
    else: 
        return False

new_shape_list = [0]
if common_member(new_shape_list, custom_list):
    print('Validation and training set overlap.')
new_shapes_data = dataset.SmalDataset(SMAL, device=DEVICE, train=False, test=False, custom=True, custom_list=new_shape_list, transform_data=False)

Compute target eigenvalues:

In [13]:
from utils.spectral import *
from utils.eigendecomposition import Eigsh_torch

k_evals      = params['eigen_isospectralization_num']
k_sub        = params['eigen_bandwitdth_num']

e_init   = np.zeros([k_evals])
e_target = np.zeros([k_evals])

epsilon = adv_ex.perturbation.epsilon.detach().cpu().numpy()

vertices_list = []
phi_list      = []
faces         = new_shapes_data[0].face.T

shape_index = 0

vertices_list.append(new_shapes_data[shape_index].pos.double())

# Compute original eigenvectors and eigenvalues for optimization.
W, _, A = calc_LB_FEM(vertices_list[shape_index], faces, DEVICE)
C = decomposition_torch(W, A)
e_all, phi_non_norm = eigsh(C.to_dense(), C.values(), C.indices(), max(k_evals, k_sub+1))

e_init = e_all[:k_evals]

A_inverse = A.rsqrt().detach().cpu().numpy()
phi_i = A_inverse[:,None] * phi_non_norm
phi_list.append(phi_i)

# Compute target eigenvaues.
e_target = e_init * (1 + epsilon)

for j in range(1, k_evals):
    if ((e_target[j] - e_target[j-1]) < 0):
      e_target = np.sort(e_target)

Optimize the new shape for eigenvalues alignment:

In [14]:
import time

## Set parameters.
MAX_ITER = 1500
L_RATE   = 1e-4
params_isosp = {'max_iter': MAX_ITER, 'l_rate': L_RATE}

vertices_final_list = []

## Target evals and evecs for optimization.
target_evals = torch.as_tensor(e_target, device=DEVICE, dtype=torch.double)
phi_sub      = torch.as_tensor(phi_list[shape_index][:,1:(k_sub+1)], device=DEVICE, dtype=torch.double)

## Initialize optimization variable and loss list.
coeff_init = torch.zeros((k_sub,3), requires_grad=False, device=DEVICE, dtype=torch.double)
coeff = coeff_init.clone().requires_grad_()

optimizer = torch.optim.Adam([coeff], lr=L_RATE) 

for it in range(MAX_ITER):   
    t = time.time()
    optimizer.zero_grad()

    ## Compute eigenvalues of perturbed shape.    
    disp = torch.mm(phi_sub, coeff)
    W, _, A = calc_LB_FEM(vertices_list[shape_index] + disp, faces, DEVICE)
    C = decomposition_torch(W, A)
    e_it = Eigsh_torch.apply(C.to_dense(), C.values(), C.indices(), k_evals)

    ## Compute and store loss.
    loss = ((e_it[1:] - target_evals[1:]) / target_evals[1:]).pow(2).sum() 

    ## Backpropagate.
    loss.backward()
    optimizer.step()

    ## Stop optimization if eigenvalues are aligned.
    if(loss < 1e-7):  
        break  

    print('shape %d -- %d] %.2e   in %.2fs'% (shape_index, it, loss.item(), time.time()-t))

perturbation = torch.mm(phi_sub, coeff)
perturbed_vertices = vertices_list[shape_index] + perturbation
vertices_final_list.append(perturbed_vertices)

e_final = e_it.clone().detach().cpu().numpy()

shape 0 -- 0] 1.02e-03   in 0.37s
shape 0 -- 1] 5.99e-04   in 0.40s
shape 0 -- 2] 3.79e-04   in 0.40s
shape 0 -- 3] 3.26e-04   in 0.39s
shape 0 -- 4] 3.56e-04   in 0.41s
shape 0 -- 5] 3.86e-04   in 0.40s
shape 0 -- 6] 3.78e-04   in 0.41s
shape 0 -- 7] 3.37e-04   in 0.41s
shape 0 -- 8] 2.85e-04   in 0.38s
shape 0 -- 9] 2.40e-04   in 0.39s
shape 0 -- 10] 2.14e-04   in 0.39s
shape 0 -- 11] 2.10e-04   in 0.39s
shape 0 -- 12] 2.19e-04   in 0.40s
shape 0 -- 13] 2.29e-04   in 0.39s
shape 0 -- 14] 2.30e-04   in 0.39s
shape 0 -- 15] 2.20e-04   in 0.39s
shape 0 -- 16] 2.01e-04   in 0.39s
shape 0 -- 17] 1.80e-04   in 0.39s
shape 0 -- 18] 1.63e-04   in 0.38s
shape 0 -- 19] 1.54e-04   in 0.40s
shape 0 -- 20] 1.53e-04   in 0.39s
shape 0 -- 21] 1.57e-04   in 0.40s
shape 0 -- 22] 1.60e-04   in 0.39s
shape 0 -- 23] 1.60e-04   in 0.38s
shape 0 -- 24] 1.55e-04   in 0.39s
shape 0 -- 25] 1.46e-04   in 0.42s
shape 0 -- 26] 1.36e-04   in 0.40s
shape 0 -- 27] 1.28e-04   in 0.39s
shape 0 -- 28] 1.24e-04   in 0

Check misclassification:

In [15]:
## Print original and final logits.
cmodel = lambda x : model(x.float())
success_list = []

for pos_i, perturbed_pos_i, shape_index in zip(vertices_list, vertices_final_list, new_shape_list):

    print('\nShape ', shape_index)

    out: torch.Tensor = cmodel(perturbed_pos_i)
    _, lbl_prediction = out.max(dim=-1)
    print('Perturbed logits : ', out.detach().cpu().numpy())

    out: torch.Tensor = cmodel(pos_i)
    _, lbl_original = out.max(dim=-1)
    print('Original logits  : ', out.detach().cpu().numpy())

    success = (lbl_prediction != lbl_original).item()
    success_list.append(success)

    print("Successful" if not int(lbl_prediction) == int(lbl_original) else "Unsuccessful",
          "[prediction: ", int(lbl_prediction),
          ", original: ", int(lbl_original), "]")


Shape  0
Perturbed logits :  [-1.9460883  -0.79826117 -3.236001    0.57171476  0.24441522]
Original logits  :  [-2.4567692  -0.55528164 -3.6982324  -0.32199374  0.926794  ]
Successful [prediction:  3 , original:  4 ]


Visualize deformed shape:

In [16]:
from utils import visualize_and_compare

for pos_i, perturbed_pos_i in zip(vertices_list, vertices_final_list):

    ## Rotate mesh for visualization.
    pos_i_rotated           = pos_i.clone()
    pos_i_rotated[:,1]      = -1 * pos_i_rotated[:,1]

    perturbed_pos_i_rotated      = perturbed_pos_i.clone()
    perturbed_pos_i_rotated[:,1] = -1 * perturbed_pos_i_rotated[:,1]

    visualize_and_compare(perturbed_pos_i_rotated, faces, pos_i_rotated, faces, (pos_i-perturbed_pos_i).norm(p=2,dim=-1))